# Miniflow
A mini tensorflow done by using graphs and raw python

In [8]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value




In [9]:
"""
You need to change the Add() class below.
"""

class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Nodes from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Nodes to which this Node passes values
        self.outbound_nodes = []
        # A calculated value
        self.value = None
        # Add this node as an outbound node on its inputs.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)

    # These will be implemented in a subclass.
    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_nodes` and
        store the result in self.value.
        """
        raise NotImplemented


class Input(Node):
    def __init__(self):
        # an Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator
        Node.__init__(self)

    # NOTE: Input node is the only node that may
    # receive its value as an argument to forward().
    #
    # All other node implementations should calculate their
    # values from the value of previous nodes, using
    # self.inbound_nodes
    #
    # Example:
    # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        if value is not None:
            self.value = value

## Add function MiniFlow

In [11]:
class Add(Node):
    def __init__(self, x, y):
        # You could access `x` and `y` in forward with
        # self.inbound_nodes[0] (`x`) and self.inbound_nodes[1] (`y`)
        Node.__init__(self, [x, y])

    def forward(self):
        """
        Set the value of this node (`self.value`) to the sum of its inbound_nodes.

        Your code here!
        """
        self.value = 0
        for inbound_node in self.inbound_nodes:
            self.value += inbound_node.value

In [12]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""

x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


10 + 5 = 15 (according to miniflow)


## Linear function MiniFlow
![alt text](neuron.png "neuron")

In [83]:
import numpy as np

inputs = [6, 14, 3]
weights = [0.5, 0.25, 1.4]
bias = 2

output = np.dot(inputs, weights) + bias
output

12.699999999999999

In [36]:
import numpy as np

class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weigahts and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodesa.

    def forward(self):
        """
        Set self.value to the value of the linear function output.

        Your code goes here!
        """
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        
        # raw solution
        #self.value = bias
        #for x, w in zip(inputs, weights):
        #    self.value += x * w
        
        # numpy solution
        self.value = np.dot(inputs, weights) + bias

In [37]:
"""
NOTE: Here we're using an Input node for more than a scalar.
In the case of weights and inputs the value of the Input node is
actually a python list!

In general, there's no restriction on the values that can be passed to an Input node.
"""

inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {
    inputs: [6, 14, 3],
    weights: [0.5, 0.25, 1.4],
    bias: 2
}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output) # should be 12.7 with this example

12.7


In [39]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)

[[-9.  4.]
 [-9.  4.]]


## Sigmoid activation function
![alt text](sigmoid_graph.png "sigmoid_graph")
![alt text](sigmoid_equation.png "sigmoid_equation")

In [48]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used later with `backward` as well.

        `x`: A numpy array-like object.

        Return the result of the sigmoid function.

        Your code here!
        """
        return 1 / (1 + np.exp(-x))


    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function, `_sigmoid`.

        Your code here!
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)


In [49]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


## Cost MSE

$$C(w,b)=\frac{1}{m}\sum_{x} ||y(x)-a||^2$$
Here w denotes the collection of all weights in the network, b all the biases, m is the total number of training examples, a is the approximation of y(x) by the network, both a and y(x) are vectors of the same length.

In [50]:
w1  = np.array([[1, 2], [3, 4]])
np.reshape(w1, -1)

array([1, 2, 3, 4])

In [69]:
def forward_pass(graph):
    """
    Performs a forward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        m = self.inbound_nodes[0].value.shape[0]

        # self.value = np.sum(np.square(y-a)) / m
        # by np.mean() -> mitja aritmètica
        self.value = np.mean((y - a)**2)


In [71]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

[[ 12.25]
 [  9.  ]
 [ 49.  ]]
23.4166666667


## Gradient Descent

```x = x - learning_rate * gradient_of_x```

x is a parameter used by the neural network (i.e. a single weight or bias).

In [74]:
def gradient_descent_update(x, gradx, learning_rate):
    """
    Performs a gradient descent update.
    """
    # TODO: Implement gradient descent.
    x = x - learning_rate * gradx
    
    # Return the new value for x
    return x

In [81]:
"""
Given the starting point of any `x` gradient descent
should be able to find the minimum value of x for the
cost function `f` defined below.
"""
import random

def f(x):
    """
    Quadratic function.

    It's easy to see the minimum value of the function
    is 5 when is x=0.
    """
    return x**2 + 5


def df(x):
    """
    Derivative of `f` with respect to `x`.
    """
    return 2*x


# Random number between 0 and 10,000. Feel free to set x whatever you like.
x = random.randint(0, 10000)
# TODO: Set the learning rate
learning_rate = 0.1
epochs = 100

for i in range(epochs+1):
    cost = f(x)
    gradx = df(x)
    print("EPOCH {}: Cost = {:.3f}, x = {:.3f}".format(i, cost, gradx))
    x = gradient_descent_update(x, gradx, learning_rate)


EPOCH 0: Cost = 12376329.000, x = 7036.000
EPOCH 1: Cost = 7920852.360, x = 5628.800
EPOCH 2: Cost = 5069347.310, x = 4503.040
EPOCH 3: Cost = 3244384.079, x = 3602.432
EPOCH 4: Cost = 2076407.610, x = 2881.946
EPOCH 5: Cost = 1328902.671, x = 2305.556
EPOCH 6: Cost = 850499.509, x = 1844.445
EPOCH 7: Cost = 544321.486, x = 1475.556
EPOCH 8: Cost = 348367.551, x = 1180.445
EPOCH 9: Cost = 222957.033, x = 944.356
EPOCH 10: Cost = 142694.301, x = 755.485
EPOCH 11: Cost = 91326.153, x = 604.388
EPOCH 12: Cost = 58450.538, x = 483.510
EPOCH 13: Cost = 37410.144, x = 386.808
EPOCH 14: Cost = 23944.292, x = 309.447
EPOCH 15: Cost = 15326.147, x = 247.557
EPOCH 16: Cost = 9810.534, x = 198.046
EPOCH 17: Cost = 6280.542, x = 158.437
EPOCH 18: Cost = 4021.347, x = 126.749
EPOCH 19: Cost = 2575.462, x = 101.399
EPOCH 20: Cost = 1650.096, x = 81.120
EPOCH 21: Cost = 1057.861, x = 64.896
EPOCH 22: Cost = 678.831, x = 51.917
EPOCH 23: Cost = 436.252, x = 41.533
EPOCH 24: Cost = 281.001, x = 33.227


## Backpropagation

![alt text](backprop.png)
![alt text](dcdw1-chain.png)
To find the gradient, you just multiply the gradients for all nodes in front of it going backwards from the cost. This is the idea behind backpropagation. The gradients are passed backwards through the network and used with gradient descent to update the weights and biases. If a node has multiple outgoing nodes, you just sum up the gradients from each node.

In [113]:
"""
Implement the backward method of the Sigmoid node.
"""
import numpy as np


class Node(object):
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes with edges into this node.
        self.inbound_nodes = inbound_nodes
        # The eventual value of this node. Set by running
        # the forward() method.
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        # Sets this node as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object.
        self.gradients = {self: 0}
        # Weights and bias may be inputs, so you need to sum
        # the gradient from output gradients.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] += grad_cost * 1


class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)


class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            """
            TODO: Your code goes here!

            Set the gradients property to the gradients with respect to each input.

            NOTE: See the Linear node and MSE node for examples.
            chain rule= it is the product from the backpropagation grad_cost (grad_cost = n.gradients[self])
            with the partial derivative!!!
            """
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += grad_cost * (sigmoid * (1 - sigmoid))


class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calculates the gradient of the cost.

        This is the final node of the network so outbound nodes
        are not a concern.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()


In [114]:
"""
Test your network here!

No need to change this code, but feel free to tweak it
to test your network!

Make your changes to backward method of the Sigmoid class
"""

X, W, b = Input(), Input(), Input()
y = Input()
f = Linear(X, W, b)
a = Sigmoid(f)
cost = MSE(y, a)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2.], [3.]])
b_ = np.array([-3.])
y_ = np.array([1, 2])

feed_dict = {
    X: X_,
    y: y_,
    W: W_,
    b: b_,
}

graph = topological_sort(feed_dict)
forward_and_backward(graph)
# return the gradients for each Input
gradients = [t.gradients[t] for t in [X, y, W, b]]

"""
Expected output

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]
"""
print(gradients)


[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]


## Stochastic Gradient Descent
Stochastic Gradient Descent (SGD) is a version of Gradient Descent where on each forward pass a batch of data is randomly sampled from total dataset. Remember when we talked about the batch size earlier? That's the size of the batch. Ideally, the entire dataset would be fed into the neural network on each forward pass, but in practice, it's not practical due to memory constraints. SGD is an approximation of Gradient Descent, the more batches processed by the neural network, the better the approximation.

A naïve implementation of SGD involves:

1. Randomly sample a batch of data from the total dataset.
2. Running the network forward and backward to calculate the gradient (with data from (1)).
3. Apply the gradient descent update. ![alt text](sgd_update.png)
4. Repeat steps 1-3 until convergence or the loop is stopped by another mechanism (i.e. the number of epochs).

In [129]:
def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # TODO: update all the `trainables` with SGD
    # You can access and assign the value of a trainable with `value` attribute.
    # Example:
    # for t in trainables:
    #   t.value = your implementation here
    for t in trainables:
        t.value = t.value - learning_rate * t.gradients[t]


In [131]:
"""
Check out the new network architecture and dataset!

Notice that the weights and biases are
generated randomly.

No need to change anything, but feel free to tweak
to test your network, play around with the epochs, batch size, etc!
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 1000
# Total number of examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        forward_and_backward(graph)

        # Step 3
        sgd_update(trainables)

        loss += graph[-1].value

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))


Total number of examples = 506
Epoch: 1, Loss: 137.786
Epoch: 2, Loss: 33.297
Epoch: 3, Loss: 26.488
Epoch: 4, Loss: 20.102
Epoch: 5, Loss: 27.499
Epoch: 6, Loss: 22.509
Epoch: 7, Loss: 17.468
Epoch: 8, Loss: 23.890
Epoch: 9, Loss: 17.394
Epoch: 10, Loss: 14.918
Epoch: 11, Loss: 15.996
Epoch: 12, Loss: 12.336
Epoch: 13, Loss: 15.849
Epoch: 14, Loss: 13.314
Epoch: 15, Loss: 10.286
Epoch: 16, Loss: 11.584
Epoch: 17, Loss: 12.928
Epoch: 18, Loss: 14.513
Epoch: 19, Loss: 9.916
Epoch: 20, Loss: 12.342
Epoch: 21, Loss: 11.001
Epoch: 22, Loss: 11.408
Epoch: 23, Loss: 10.584
Epoch: 24, Loss: 11.644
Epoch: 25, Loss: 7.105
Epoch: 26, Loss: 9.895
Epoch: 27, Loss: 10.794
Epoch: 28, Loss: 9.743
Epoch: 29, Loss: 8.583
Epoch: 30, Loss: 8.638
Epoch: 31, Loss: 9.257
Epoch: 32, Loss: 9.668
Epoch: 33, Loss: 9.507
Epoch: 34, Loss: 7.237
Epoch: 35, Loss: 9.732
Epoch: 36, Loss: 8.582
Epoch: 37, Loss: 8.429
Epoch: 38, Loss: 8.887
Epoch: 39, Loss: 8.433
Epoch: 40, Loss: 8.756
Epoch: 41, Loss: 8.978
Epoch: 42,

Epoch: 348, Loss: 4.320
Epoch: 349, Loss: 4.411
Epoch: 350, Loss: 3.650
Epoch: 351, Loss: 3.876
Epoch: 352, Loss: 4.418
Epoch: 353, Loss: 3.915
Epoch: 354, Loss: 4.345
Epoch: 355, Loss: 4.279
Epoch: 356, Loss: 4.974
Epoch: 357, Loss: 4.248
Epoch: 358, Loss: 4.101
Epoch: 359, Loss: 3.969
Epoch: 360, Loss: 4.410
Epoch: 361, Loss: 3.826
Epoch: 362, Loss: 3.793
Epoch: 363, Loss: 4.228
Epoch: 364, Loss: 4.685
Epoch: 365, Loss: 3.930
Epoch: 366, Loss: 4.552
Epoch: 367, Loss: 3.643
Epoch: 368, Loss: 3.662
Epoch: 369, Loss: 3.854
Epoch: 370, Loss: 3.924
Epoch: 371, Loss: 4.586
Epoch: 372, Loss: 3.769
Epoch: 373, Loss: 4.529
Epoch: 374, Loss: 4.261
Epoch: 375, Loss: 3.812
Epoch: 376, Loss: 3.996
Epoch: 377, Loss: 4.581
Epoch: 378, Loss: 4.330
Epoch: 379, Loss: 3.846
Epoch: 380, Loss: 4.048
Epoch: 381, Loss: 3.554
Epoch: 382, Loss: 4.172
Epoch: 383, Loss: 3.790
Epoch: 384, Loss: 3.906
Epoch: 385, Loss: 4.289
Epoch: 386, Loss: 3.934
Epoch: 387, Loss: 4.226
Epoch: 388, Loss: 3.740
Epoch: 389, Loss

Epoch: 708, Loss: 3.874
Epoch: 709, Loss: 3.690
Epoch: 710, Loss: 3.903
Epoch: 711, Loss: 3.638
Epoch: 712, Loss: 3.792
Epoch: 713, Loss: 3.508
Epoch: 714, Loss: 3.843
Epoch: 715, Loss: 3.777
Epoch: 716, Loss: 3.413
Epoch: 717, Loss: 4.111
Epoch: 718, Loss: 3.549
Epoch: 719, Loss: 3.575
Epoch: 720, Loss: 3.621
Epoch: 721, Loss: 3.744
Epoch: 722, Loss: 3.259
Epoch: 723, Loss: 3.230
Epoch: 724, Loss: 3.368
Epoch: 725, Loss: 3.107
Epoch: 726, Loss: 3.931
Epoch: 727, Loss: 4.047
Epoch: 728, Loss: 3.738
Epoch: 729, Loss: 4.009
Epoch: 730, Loss: 3.983
Epoch: 731, Loss: 3.889
Epoch: 732, Loss: 3.621
Epoch: 733, Loss: 3.367
Epoch: 734, Loss: 3.821
Epoch: 735, Loss: 3.281
Epoch: 736, Loss: 3.339
Epoch: 737, Loss: 3.506
Epoch: 738, Loss: 3.752
Epoch: 739, Loss: 3.363
Epoch: 740, Loss: 3.300
Epoch: 741, Loss: 3.550
Epoch: 742, Loss: 3.531
Epoch: 743, Loss: 3.683
Epoch: 744, Loss: 3.267
Epoch: 745, Loss: 3.796
Epoch: 746, Loss: 3.827
Epoch: 747, Loss: 3.702
Epoch: 748, Loss: 3.726
Epoch: 749, Loss